# Provisioning Federated learning (FL) Tool 

FL has been simplified in V3.1 to have a provisioning tool that allows admins to:
- Configure FL experiment
- Send startup packages to FL clients (password protected zip file)

By the end of this notebook you would be able to provision an FL experiment and start the server.   


## Prerequisites
- Running this notebook from within clara docker following setup in [readMe.md](../../readMe.md)
- Provisioning doesn't require GPUs. 
   

### Resources
You could watch the free GTC 2020 talks covering Clara Train SDK 
- [Clara Developer Day: Federated Learning using Clara Train SDK](https://developer.nvidia.com/gtc/2020/video/S22564)
 

## DataSet 
No dataset is used in this notebook


# Lets get started
Cell below defines functions that we will use throughout the notebook

In [ ]:
def listDirs(newMMARDir):
    !ls $newMMARDir
    !echo ----config
    !ls $newMMARDir/config
    !echo ----commands
    !ls $newMMARDir/commands
def printFile(filePath,lnSt,lnOffset):
    print ("showing ",str(lnOffset)," lines from file ",filePath, "starting at line",str(lnSt))
    lnOffset=lnSt+lnOffset
    !< $filePath head -n "$lnOffset" | tail -n +"$lnSt"   


# Provisioning Components
The provisioning tool is the first step to configure a FL experiment. 
This consists of creating: 
1. Project yaml file, which defines: project name, participants, server name and other settings
2. Authorization json file which defines: groups, roles, rights. 
3. Run provisioning tool  

### 1. UI tool to generate project.yaml and authorization.json
We have developed a simple html page that would generate the project.yaml and authorization.json files for you.
Simply open the html or run cell below to see the page. 
You would need to:
- Change the servername.
- Add/remove groups.
- Add/remove polices
- Add/remove users 
- Click `Generate artifacts`
- Click download or copy / past the files as new yaml and json files    


In [ ]:
import IPython
IPython.display.IFrame('./FLprovUI.html',width=850,height=700)

### 2 Run Provisioning tool 
For simplicity we have included a project1.yaml and project1auth.json files for you to use in this notebook.
In order to see their content simply run cell below 

In [ ]:
MMAR_ROOT="/claraDevDay/FL/"
PROV_DIR="provisioning"
PROJ_NAME="project1"

printFile(MMAR_ROOT+PROJ_NAME+".yml",0,50)
print("---------------------")
printFile(MMAR_ROOT+PROJ_NAME+"auth.json",0,200)


### 2.1 Run provisioning tool
Cell below show help on how to use the cli for the provisioning tool 

In [ ]:
!provision -h

In [ ]:
%cd $MMAR_ROOT
!rm -r $PROJ_NAME
%mkdir -p $PROJ_NAME/$PROV_DIR
# !ln -s /opt/nvidia/medical/tools/project.yml $MMAR_ROOT/project.yml

In [ ]:
PROJ_PATH=MMAR_ROOT+PROJ_NAME+"/"
PROV_PATH=PROJ_PATH+PROV_DIR+"/"
%cd $PROJ_PATH
!provision -p $MMAR_ROOT/$PROJ_NAME'.yml' -o $PROV_DIR -t $PROV_PATH/audit.pkl -a $MMAR_ROOT/$PROJ_NAME'auth.json'


### 3. Send startup kits to participants
In a real experiment, you would send packages to each site so they would run it on their system. 
Here we would extract and simulate a server, 3 clients and an admin all in this tutorial. 

Cell above should have printed out passwords for each package. 
You should replace the password from above cell to the corresponding file in cell below 

In [ ]:
%cd  $PROV_PATH
server,client1,client2,client3,client4="server","client1","client2","client3","client4"
admin,leadIT,siteResearch,leadITSec="admin","leadIT","siteResearch","leadITSec"
!unzip -oP Gt70p3kYKoIVfM48 server.zip -d ../$server
!unzip -oP E9HCjgF6VBMoALrU client1.zip -d ../$client1
!unzip -oP mXoq4RdhItNuDvPe client2.zip -d ../$client2
!unzip -oP E9HCjgF6VBMoALrU client3.zip -d ../$client3
!unzip -oP E9HCjgF6VBMoALrU client4.zip -d ../$client4
!unzip -oP ecpUmT10J0WDhsKu admin@admin.com.zip -d ../$admin
!unzip -oP ecpUmT10J0WDhsKu leadIT@org1.com.zip -d ../$leadIT
!unzip -oP ecpUmT10J0WDhsKu siteresearcher@org2.com.zip -d ../$siteResearch
!unzip -oP ecpUmT10J0WDhsKu leadIT@secure.com.zip -d ../$leadITSec

### 4. Double check on rights before sending kits to clients

As you may notice creating users and rights could be a bit tricky with large number of clients (20+).
Specially if each client has different requirements. 
In order to make this task a bit easier, 
we have created a checking tool where you pass the authorization.json, 
then check that it is doing what you intended to do.

In order to validate this you can use the shell tool as 
```
cd /claraDevDay/FL/project1/server/startup
authz_preview -p authorization.json
```  

This would open an interactive prompt. You can type `?` to see list of commands as 

```
>?

Documented commands (type help <topic>):
========================================
bye  help

Undocumented commands:
======================
eval_right  eval_user    show_rights  show_site_rules  show_users
eval_rule   show_config  show_rules   show_sites     
```


You can use `show_users` to find all users. 
Then in order to see rules for a certain site you can type
`eval_user <user> <org name>`
you should see something similar to 
```
-------------------------
| RIGHT        | RESULT |
-------------------------
| upload_mmar  | True   |
| deploy_self  | False  |
| deploy_all   | False  |
| train_self   | False  |
| train_all    | False  |
| view_self    | False  |
| view_all     | False  |
| operate_all  | False  |
| operate_self | False  |
-------------------------
```

In our sample client4 doens't allow anyone to upload any mmars. You can check by 
`eval_user leadIT@org1.com client4`
vs `eval_user leadIT@org1.com client1`

but user `leadIT@secure.com` can upload to client4. You can check by 
`eval_user leadIT@secure.com client4`

You could also verify only one specific right using 
```
eval_right leadIT@org1.com deploy_self client4
```


# Next steps 
1. You should run the [admin notebook](Admin.ipynb) to start your FL experiment.
2. You may want to take a look at the [client notebook](Client.ipynb)



# Exercise 

1. Provision your own FL experiment. For this you should 
    1. got back to cell2 and which displays the FLprovUI.html page 
    2. Make changes modify it with your client names and emails
    2. Get your server host name using any of the commands below 
    ```
    hostname
    hostnamectl
    cat /proc/sys/kernel/hostname
    ```
    3. In the yml file change the cn name to the server name as 
    ```
    server:
      cn: <your server name >
    ``` 
    4. Click generate artifacts 
    5. Save `provision.yml` and `authz_config.json` for your new project 
    6. Continue steps in the notebook 